In [1]:
import requests
import json
import pandas as pd

from cities import city_data
from weather import beaufort

In [2]:
# API Key
key = "329feaeca8c343abb9845244a3ccc8d8"

# Forecat days
days = 14

In [3]:
# initialize score matrix+
score_matrix = pd.DataFrame()

for city_name in city_data.keys():
    lat, long = city_data[city_name]['lat'], city_data[city_name]['long']
    URL = "http://api.weatherbit.io/v2.0/forecast/daily?lat={}&lon={}&key={}&days={}".format(lat, long, key, days)
    r = requests.get(URL)
    files = r.json()
    data = pd.DataFrame.from_dict(files.get("data"))

    # Get Cloud Cover
    cc = data["clouds"]

    # Get visibility
    fm = data["vis"]

    # Get precipitation
    pr = data["precip"]/24

    # Get wind speed
    w = data["wind_spd"]

    w = pd.DataFrame(beaufort(w))

    # Initialise score with 10, then deduct
    s = 10

    scores = []

    for i in range(days):
        s = 10
    
        # Cloud Cover (cloud cover in daytime)
        if (cc.iloc[i] > 1/8) and (cc.iloc[i] <= 5/8):
            s -= 1
        elif (cc.iloc[i] > 5/8) and (cc.iloc[i] <= 7/8):
            s -= 2
        elif (cc.iloc[i] > 7/8):
            s -= 3

        # Fog and mist (in hours)
        # From https://en.wikipedia.org/wiki/Visibility#Fog,_mist,_haze,_and_freezing_drizzle
        # fog: a visibility of less than 1 kilometre
        # mist: a visibility of between 1 kilometre (0.62 mi) and 2 kilometres (1.2 mi)
        # haze: a visibility from 2 kilometres (1.2 mi) to 5 kilometres (3.1 mi)

        if fm.iloc[i] < 1:
            s -= 2
        elif fm.iloc[i] <= 2:
            s -= 1

        # Precipitation (duration of expected rainfall in minutes)

        # From https://de.wikipedia.org/wiki/Regen
        # light rain = 0,1 to 0,5 mm/h
        # moderate rain = 0,5 to 4 mm/h
        # heavy rain = 4 to 10 mm/h

        # Get prec / h
        if (pr.iloc[i] >= 0.1) and (pr.iloc[i] <= 0.5):
            s -= 1
        elif (pr.iloc[i] > 0.5) and (pr.iloc[i] <= 4):
            s -= 2
        elif (pr.iloc[i] > 4) and (pr.iloc[i] <= 10):
            s -= 3
        elif pr.iloc[i] > 10:
            s -= 4

        # Wind (daytime maximum wind strenght (average of 10 minutes) in Bft)
        # if w can not be retrieved in Bft:
        # convert m/s into Bft: https://www.windfinder.com/wind/windspeed.htm

        if (w.iloc[i][0] >= 2) & (w.iloc[i][0] < 4):
            s -= 1
        elif (w.iloc[i][0] >= 4) & (w.iloc[i][0] < 6): 
            s -= 2
        elif (w.iloc[i][0] > 6):
            s -= 3

        # Set 1 as lowest score
        if s < 1:
            s = 1
        
        scores.append(s)

    scores = pd.DataFrame(scores)
    score_matrix = pd.concat([score_matrix,scores], axis=1)

score_matrix

,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,4,4,4,4,4,4,4,4,4,4,...,4,4,8,7,4,4,4,5,4,4
1,4,4,3,4,4,4,4,4,4,4,...,4,4,5,3,4,4,4,5,4,4
2,5,5,4,4,4,4,4,5,4,4,...,4,5,4,4,4,4,4,4,5,4
3,4,5,4,4,5,5,5,4,6,4,...,5,5,4,4,4,4,4,5,5,4
4,6,7,6,6,9,10,7,6,6,10,...,6,6,9,9,5,6,9,9,7,7
5,6,7,6,6,6,6,6,6,5,7,...,5,5,6,7,6,6,6,6,6,6
6,5,5,6,9,5,4,6,5,6,6,...,5,5,6,9,6,6,6,6,6,6
7,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,5,6,6,5,9,6
8,9,9,6,6,6,6,9,9,7,6,...,6,6,6,6,6,6,6,6,6,9
9,9,6,9,6,10,6,6,9,6,9,...,6,6,6,9,9,9,9,6,6,6


In [4]:
score_matrix.columns = city_data.keys()

In [5]:
score_matrix

,Munich,Zurich,Montpellier,Barcelona,Lyon,Geneva,Colmar,Avignon,Ceret,Mende,...,Chamonix,St.Moritz,Genoa,Marseille,Bayonne,Bordeaux,Saint Etienne,Milan,Paris,Strasbourg
0,4,4,4,4,4,4,4,4,4,4,...,4,4,8,7,4,4,4,5,4,4
1,4,4,3,4,4,4,4,4,4,4,...,4,4,5,3,4,4,4,5,4,4
2,5,5,4,4,4,4,4,5,4,4,...,4,5,4,4,4,4,4,4,5,4
3,4,5,4,4,5,5,5,4,6,4,...,5,5,4,4,4,4,4,5,5,4
4,6,7,6,6,9,10,7,6,6,10,...,6,6,9,9,5,6,9,9,7,7
5,6,7,6,6,6,6,6,6,5,7,...,5,5,6,7,6,6,6,6,6,6
6,5,5,6,9,5,4,6,5,6,6,...,5,5,6,9,6,6,6,6,6,6
7,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,5,6,6,5,9,6
8,9,9,6,6,6,6,9,9,7,6,...,6,6,6,6,6,6,6,6,6,9
9,9,6,9,6,10,6,6,9,6,9,...,6,6,6,9,9,9,9,6,6,6


In [6]:
score_matrix.to_csv('city_weather_scores.csv')